In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [11]:
raw_data= pd.read_csv('arcos-ms-statewide-itemized.tsv', sep = '\t')

In [12]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2275035 entries, 0 to 2275034
Data columns (total 42 columns):
REPORTER_DEA_NO          object
REPORTER_BUS_ACT         object
REPORTER_NAME            object
REPORTER_ADDL_CO_INFO    object
REPORTER_ADDRESS1        object
REPORTER_ADDRESS2        object
REPORTER_CITY            object
REPORTER_STATE           object
REPORTER_ZIP             int64
REPORTER_COUNTY          object
BUYER_DEA_NO             object
BUYER_BUS_ACT            object
BUYER_NAME               object
BUYER_ADDL_CO_INFO       object
BUYER_ADDRESS1           object
BUYER_ADDRESS2           object
BUYER_CITY               object
BUYER_STATE              object
BUYER_ZIP                int64
BUYER_COUNTY             object
TRANSACTION_CODE         object
DRUG_CODE                int64
NDC_NO                   object
DRUG_NAME                object
QUANTITY                 float64
UNIT                     float64
ACTION_INDICATOR         object
ORDER_FORM_NO           

In [13]:
raw_data.head(5)

,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,...,DOSAGE_UNIT,TRANSACTION_ID,Product_Name,Ingredient_Name,Measure,MME_Conversion_Factor,Combined_Labeler_Name,Revised_Company_Name,Reporter_family,dos_str
0,PB0034861,DISTRIBUTOR,BELLCO DRUG CORP,NaN,5500 NEW HORIZONS BLVD,NaN,NORTH AMITYVILLE,NY,11701,SUFFOLK,...,100.0,1927,HYDROCODONE BIT./IBUPROFEN;7.5MG & 2,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,"Teva Pharmaceuticals USA, Inc.",Teva,Bellco Drug Corp,7.5
1,PB0034861,DISTRIBUTOR,BELLCO DRUG CORP,NaN,5500 NEW HORIZONS BLVD,NaN,NORTH AMITYVILLE,NY,11701,SUFFOLK,...,100.0,8958,HYDROCODONE BIT/ACETA 7.5MG/325MG US,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,SpecGx LLC,Mallinckrodt,Bellco Drug Corp,7.5
2,PB0034861,DISTRIBUTOR,BELLCO DRUG CORP,NaN,5500 NEW HORIZONS BLVD,NaN,NORTH AMITYVILLE,NY,11701,SUFFOLK,...,500.0,675,"HYDROCODONE.BIT./ACET.,10MG & 325MG/",HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Par Pharmaceutical,"Endo Pharmaceuticals, Inc.",Bellco Drug Corp,10.0
3,PB0034861,DISTRIBUTOR,BELLCO DRUG CORP,NaN,5500 NEW HORIZONS BLVD,NaN,NORTH AMITYVILLE,NY,11701,SUFFOLK,...,1500.0,1393,HYDROCODONE/APAP 7.5MG/200MG TABS.,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Par Pharmaceutical,"Endo Pharmaceuticals, Inc.",Bellco Drug Corp,7.5
4,PB0167127,DISTRIBUTOR,BLOODWORTH WHOLESALE DRUGS,NaN,2128 YANK LAMB DRIVE,P.O. BOX 1849,TIFTON,GA,31793,TIFT,...,1000.0,3756,HYDROCODONE BITARTRATE & ACETA 7.5MG,HYDROCODONE BITARTRATE HEMIPENTAHYDRATE,TAB,1.0,Par Pharmaceutical,"Endo Pharmaceuticals, Inc.",Bloodworth Wholesale Drugs,7.5


In [14]:
import pandas_profiling as pp
pp.ProfileReport(raw_data)

Number of variables,42
Number of observations,2275035
Total Missing (%),18.5%
Total size in memory,729.0 MiB
Average record size in memory,336.0 B
Numeric,10
Categorical,26
Boolean,2
Date,0
Text (Unique),0
Rejected,4


In [15]:
raw_data.drop(columns =['ACTION_INDICATOR','UNIT','TRANSACTION_CODE'], inplace = True)

In [34]:
raw_data.groupby('BUYER_COUNTY').QUANTITY.sum().sort_values(ascending=False)

BUYER_COUNTY
HARRISON           392152.0
HINDS              245911.0
DESOTO             244845.0
FORREST            242637.0
JACKSON            237073.0
RANKIN             230549.0
LAUDERDALE         164663.0
MADISON            163277.0
LEE                146365.0
JONES              132072.0
PEARL RIVER        102883.0
MARION             100570.0
ADAMS               94645.0
PIKE                78834.0
LOWNDES             78517.0
ALCORN              73289.0
WARREN              71420.0
WASHINGTON          69734.0
HANCOCK             67114.0
NESHOBA             66760.0
ITAWAMBA            62474.0
LINCOLN             60975.0
SIMPSON             57935.0
UNION               48240.0
WAYNE               48135.0
MONROE              45994.0
TISHOMINGO          45374.0
LAFAYETTE           44823.0
LEFLORE             42363.0
PANOLA              42230.0
                     ...   
YAZOO               20608.0
CLARKE              19993.0
NEWTON              19189.0
WINSTON             18863.0
WEBSTER

In [16]:
# import libraries
import urllib.request
from bs4 import BeautifulSoup
# specify the url
urlpage = 'https://www.mississippi-demographics.com/counties_by_population' 
page = urllib.request.urlopen(urlpage)
soup = BeautifulSoup(page, 'html.parser')

In [17]:
soup.title

<title>Mississippi Counties by Population </title>

In [18]:
# find results within table
table = soup.find('table', class_ = "ranklist span8")
tablerow = table.findAll('tr')
counties = tablerow[:-1]
print('Number of results', len(counties))

Number of results 83


In [72]:
# Get county population
# create and write headers to a list 
Rank =[]
County=[]
Population=[]

for row in counties:
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        Rank.append(cells[0].find(text=True))
        County.append(cells[1].find('a').get('href'))
        Population.append(cells[2].find(text=True))
        
df_county=pd.DataFrame()
df_county['Rank']=Rank
df_county['County']=County
df_county['County'] = df_county.County.str.replace('-demographics','').str.replace('-county','').str.replace('-',' ').str.upper()
df_county['Population']=Population
df_county['Population']=df_county['Population'].str.replace(',','').astype(int)
df_county = df_county.set_index('Rank')




,County,Population
Rank,,
1,HINDS,237085
2,HARRISON,206650
3,DESOTO,182001
4,RANKIN,153902
5,JACKSON,143277
6,MADISON,105630
7,LEE,85202
8,LAUDERDALE,75317
9,FORREST,75036


In [67]:
#Check for equality
for i in df_county.County.values:
    if i not in raw_data.BUYER_COUNTY.unique():
        print(i)
for i in raw_data.BUYER_COUNTY.unique():
    if i not in df_county.County.values:
        print(i)
print(len(df_county.County.values))
print(len(raw_data.BUYER_COUNTY.unique()))

ISSAQUENA
82
81


In [78]:
grouped = pd.concat([raw_data.groupby('BUYER_COUNTY').QUANTITY.sum(),df_county.reset_index(drop=True)], sort= True)
grouped

,0,County,Population
ADAMS,94645.0,NaN,NaN
ALCORN,73289.0,NaN,NaN
AMITE,10480.0,NaN,NaN
ATTALA,27050.0,NaN,NaN
BENTON,3407.0,NaN,NaN
BOLIVAR,30874.0,NaN,NaN
CALHOUN,25004.0,NaN,NaN
CARROLL,1353.0,NaN,NaN
CHICKASAW,36967.0,NaN,NaN
CHOCTAW,11211.0,NaN,NaN


In [20]:
urlpage = 'https://www.mississippi-demographics.com/cities_by_population' 
page = urllib.request.urlopen(urlpage)
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_ = "ranklist span8")
tablerow = table.findAll('tr')
results = tablerow[:-1]
print('Number of results', len(results))

Number of results 347


In [22]:
# create and write headers to a list 
Rank =[]
City1=[]
City2=[]
Population=[]
for row in results:
    cells = row.findAll('td')
    states=row.findAll('th') #To store second column data
    if len(cells)==3: #Only extract table body not heading
        Rank.append(cells[0].find(text=True))
        Population.append(cells[2].find(text=True))
    if len(cells)==3 and results.index(row) < 101:
        City1.append(cells[1].find('a').get('href'))
    if len(cells)==3 and results.index(row)>=101:
        City2.append(cells[1].find(text=True))

df1 = pd.DataFrame()
df2 = pd.DataFrame()
df1['city1'] = City1
df2['city2'] = City2 
df1.city1 = df1.city1.str.replace('-demographics','').str.upper()
df2.city2 = df2.city2.str.strip('\n').str.rpartition('\n')[0].str.upper()
df3 = pd.concat([df1.city1,df2.city2])
df3 = df3.reset_index(drop=True)
df_city=pd.DataFrame()
df_city['Rank']=Rank
df_city['City']=df3
df_city['Population']=Population
df_city = df_city.set_index('Rank')
df_city

,City,Population
Rank,,
1,JACKSON,"164,422"
2,GULFPORT,"71,870"
3,SOUTHAVEN,"54,944"
4,BILOXI,"45,968"
5,HATTIESBURG,"45,951"
6,TUPELO,"38,206"
7,OLIVE-BRANCH,"38,149"
8,MERIDIAN,"37,325"
9,GREENVILLE,"29,898"
